In [2]:
from google.colab import drive

drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
%matplotlib inline
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

from sklearn.preprocessing import OrdinalEncoder
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split
from sklearn import preprocessing
from sklearn.model_selection import cross_val_score, GridSearchCV
from sklearn.metrics import confusion_matrix, accuracy_score, precision_score, recall_score, f1_score, roc_auc_score, classification_report, precision_recall_curve
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier

In [4]:
data = pd.read_csv(r"/content/drive/MyDrive/Colab Notebooks/ML homework/Copy of default of credit card clients.csv")

In [5]:
def warn(*args, **kwargs):
    pass
import warnings
warnings.warn = warn

In [6]:
data.drop('ID', axis=1)

,LIMIT_BAL,SEX,EDUCATION,MARRIAGE,AGE,PAY_0,PAY_2,PAY_3,PAY_4,PAY_5,PAY_6,BILL_AMT1,BILL_AMT2,BILL_AMT3,BILL_AMT4,BILL_AMT5,BILL_AMT6,PAY_AMT1,PAY_AMT2,PAY_AMT3,PAY_AMT4,PAY_AMT5,PAY_AMT6,default payment next month
0,20000,2,2,1,24,2,2,-1,-1,-2,-2,3913,3102,689,0,0,0,0,689,0,0,0,0,1
1,120000,2,2,2,26,-1,2,0,0,0,2,2682,1725,2682,3272,3455,3261,0,1000,1000,1000,0,2000,1
2,90000,2,2,2,34,0,0,0,0,0,0,29239,14027,13559,14331,14948,15549,1518,1500,1000,1000,1000,5000,0
3,50000,2,2,1,37,0,0,0,0,0,0,46990,48233,49291,28314,28959,29547,2000,2019,1200,1100,1069,1000,0
4,50000,1,2,1,57,-1,0,-1,0,0,0,8617,5670,35835,20940,19146,19131,2000,36681,10000,9000,689,679,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
29995,220000,1,3,1,39,0,0,0,0,0,0,188948,192815,208365,88004,31237,15980,8500,20000,5003,3047,5000,1000,0
29996,150000,1,3,2,43,-1,-1,-1,-1,0,0,1683,1828,3502,8979,5190,0,1837,3526,8998,129,0,0,0
29997,30000,1,2,2,37,4,3,2,-1,0,0,3565,3356,2758,20878,20582,19357,0,0,22000,4200,2000,3100,1
29998,80000,1,3,1,41,1,-1,0,0,0,-1,-1645,78379,76304,52774,11855,48944,85900,3409,1178,1926,52964,1804,1


# Helper Function

In [7]:
def preprocessing_(data):
    # get rid of missing values in MARRIAGE column
    data = data[data["MARRIAGE"] != 0]   
    
    # get rid of missing values in EDUCATION column
    data = data[data["EDUCATION"] != 0]    
    
    # discritize the AGE attribute to 6 interval
    data["AGE"] = pd.cut(data["AGE"], 6, labels=["AGE20", "AGE30", "AGE40", "AGE50", "AGE60", "AGE70"] )    
    #ordinal encoding
    encoder = OrdinalEncoder()
    # transform data
    data["AGE"] = encoder.fit_transform(np.array(data["AGE"]).reshape(-1, 1))
    
    return data

def print_cm(cm, labels, hide_zeroes=False, hide_diagonal=False, hide_threshold=None):
    """pretty print for confusion matrixes"""
    columnwidth = max([len(x) for x in labels] + [5])  # 5 is value length
    empty_cell = " " * columnwidth
    # Print header
    print("    " + empty_cell, end=" ")
    for label in labels:
        print("%{0}s".format(columnwidth) % label, end=" ")
    print()
    # Print rows
    for i, label1 in enumerate(labels):
        print("    %{0}s".format(columnwidth) % label1, end=" ")
        for j in range(len(labels)):
            cell = "%{0}.1f".format(columnwidth) % cm[i, j]
            if hide_zeroes:
                cell = cell if float(cm[i, j]) != 0 else empty_cell
            if hide_diagonal:
                cell = cell if i != j else empty_cell
            if hide_threshold:
                cell = cell if cm[i, j] > hide_threshold else empty_cell
            print(cell, end=" ")
        print()

        
def train_evaluate_val(model, x_train, y_train, folds, metrics):
    for metric in metrics:
        scores = cross_val_score(model, x_train, y_train, cv=folds, scoring = metric)
        print(f'{metric}:')
        print(f'Cross-validation: {scores}')
        print(f"Average: {scores.mean()}")
        print("----------------------------------------------------------")
    
def test_evaluate(model, x_test, y_test):
    prediction = model.predict(x_test)
    # prediction_prob = model.predict_proba(x_test)
    confusion = confusion_matrix(y_test, prediction)
    print("Confusion matrix",print_cm(confusion, ['Not pay', 'pay']))
    print('Accuracy: {:.2f}'.format(accuracy_score(y_test, prediction)))
    print('Precision: {:.2f}'.format(precision_score(y_test, prediction)))
    print('Recall: {:.2f}'.format(recall_score(y_test, prediction)))
    print('F1: {:.2f}'.format(f1_score(y_test, prediction)))
    print('AUC: {:.2f}'.format(roc_auc_score(y_test, prediction)))
    print(classification_report(y_test, prediction, target_names=['not pay', 'pay']))
    # precision_recall_curve(y_test, prediction_prob)

def optimize_model(model, x_train, y_train, parameters, metrics):
    for metric in metrics:
        print(f'{metric}:')
        grid_clf = GridSearchCV(model, param_grid = parameters, scoring = metric)
        grid_clf.fit(x_train, y_train)
        best_parameter = grid_clf.best_params_
        best_score = grid_clf.best_score_
        print(f"The best parameters: {best_parameter}\nThe best score: {best_score}\n")

# Prepare the data:

In [8]:
data = preprocessing_(data)

In [9]:
data

,ID,LIMIT_BAL,SEX,EDUCATION,MARRIAGE,AGE,PAY_0,PAY_2,PAY_3,PAY_4,PAY_5,PAY_6,BILL_AMT1,BILL_AMT2,BILL_AMT3,BILL_AMT4,BILL_AMT5,BILL_AMT6,PAY_AMT1,PAY_AMT2,PAY_AMT3,PAY_AMT4,PAY_AMT5,PAY_AMT6,default payment next month
0,1,20000,2,2,1,0.0,2,2,-1,-1,-2,-2,3913,3102,689,0,0,0,0,689,0,0,0,0,1
1,2,120000,2,2,2,0.0,-1,2,0,0,0,2,2682,1725,2682,3272,3455,3261,0,1000,1000,1000,0,2000,1
2,3,90000,2,2,2,1.0,0,0,0,0,0,0,29239,14027,13559,14331,14948,15549,1518,1500,1000,1000,1000,5000,0
3,4,50000,2,2,1,1.0,0,0,0,0,0,0,46990,48233,49291,28314,28959,29547,2000,2019,1200,1100,1069,1000,0
4,5,50000,1,2,1,3.0,-1,0,-1,0,0,0,8617,5670,35835,20940,19146,19131,2000,36681,10000,9000,689,679,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
29995,29996,220000,1,3,1,1.0,0,0,0,0,0,0,188948,192815,208365,88004,31237,15980,8500,20000,5003,3047,5000,1000,0
29996,29997,150000,1,3,2,2.0,-1,-1,-1,-1,0,0,1683,1828,3502,8979,5190,0,1837,3526,8998,129,0,0,0
29997,29998,30000,1,2,2,1.0,4,3,2,-1,0,0,3565,3356,2758,20878,20582,19357,0,0,22000,4200,2000,3100,1
29998,29999,80000,1,3,1,2.0,1,-1,0,0,0,-1,-1645,78379,76304,52774,11855,48944,85900,3409,1178,1926,52964,1804,1


# Split the data:

In [10]:
y = data["default payment next month"]
data = data.drop("default payment next month", axis=1)

x_train, x_test, y_train, y_test = train_test_split(data, y, test_size=0.2, random_state=42)

Normalization:

In [11]:
scaler2 = preprocessing.StandardScaler().fit(np.array(x_train["LIMIT_BAL"]).reshape(-1, 1))
x_train["LIMIT_BAL"] = scaler2.transform(np.array(x_train["LIMIT_BAL"]).reshape(-1, 1))
x_test["LIMIT_BAL"] = scaler2.transform(np.array(x_test["LIMIT_BAL"]).reshape(-1, 1))

scaler3 = preprocessing.StandardScaler().fit(np.array(x_train["BILL_AMT1"]).reshape(-1, 1))
x_train["BILL_AMT1"] = scaler3.transform(np.array(x_train["BILL_AMT1"]).reshape(-1, 1))
x_train["BILL_AMT2"] = scaler3.transform(np.array(x_train["BILL_AMT2"]).reshape(-1, 1))
x_train["BILL_AMT3"] = scaler3.transform(np.array(x_train["BILL_AMT3"]).reshape(-1, 1))
x_train["BILL_AMT4"] = scaler3.transform(np.array(x_train["BILL_AMT4"]).reshape(-1, 1))
x_train["BILL_AMT5"] = scaler3.transform(np.array(x_train["BILL_AMT5"]).reshape(-1, 1))
x_train["BILL_AMT6"] = scaler3.transform(np.array(x_train["BILL_AMT6"]).reshape(-1, 1))

x_test["BILL_AMT1"] = scaler2.transform(np.array(x_test["BILL_AMT1"]).reshape(-1, 1))
x_test["BILL_AMT2"] = scaler2.transform(np.array(x_test["BILL_AMT2"]).reshape(-1, 1))
x_test["BILL_AMT3"] = scaler2.transform(np.array(x_test["BILL_AMT3"]).reshape(-1, 1))
x_test["BILL_AMT4"] = scaler2.transform(np.array(x_test["BILL_AMT4"]).reshape(-1, 1))
x_test["BILL_AMT5"] = scaler2.transform(np.array(x_test["BILL_AMT5"]).reshape(-1, 1))
x_test["BILL_AMT6"] = scaler2.transform(np.array(x_test["BILL_AMT6"]).reshape(-1, 1))

scaler4 = preprocessing.StandardScaler().fit(np.array(x_train["PAY_AMT1"]).reshape(-1, 1))
x_train["PAY_AMT1"] = scaler4.transform(np.array(x_train["PAY_AMT1"]).reshape(-1, 1))
x_train["PAY_AMT2"] = scaler4.transform(np.array(x_train["PAY_AMT2"]).reshape(-1, 1))
x_train["PAY_AMT3"] = scaler4.transform(np.array(x_train["PAY_AMT3"]).reshape(-1, 1))
x_train["PAY_AMT4"] = scaler4.transform(np.array(x_train["PAY_AMT4"]).reshape(-1, 1))
x_train["PAY_AMT5"] = scaler4.transform(np.array(x_train["PAY_AMT5"]).reshape(-1, 1))
x_train["PAY_AMT6"] = scaler4.transform(np.array(x_train["PAY_AMT6"]).reshape(-1, 1))

x_test["PAY_AMT1"] = scaler4.transform(np.array(x_test["PAY_AMT1"]).reshape(-1, 1))
x_test["PAY_AMT2"] = scaler4.transform(np.array(x_test["PAY_AMT2"]).reshape(-1, 1))
x_test["PAY_AMT3"] = scaler4.transform(np.array(x_test["PAY_AMT3"]).reshape(-1, 1))
x_test["PAY_AMT4"] = scaler4.transform(np.array(x_test["PAY_AMT4"]).reshape(-1, 1))
x_test["PAY_AMT5"] = scaler4.transform(np.array(x_test["PAY_AMT5"]).reshape(-1, 1))
x_test["PAY_AMT6"] = scaler4.transform(np.array(x_test["PAY_AMT6"]).reshape(-1, 1))

In [12]:
x_train

,ID,LIMIT_BAL,SEX,EDUCATION,MARRIAGE,AGE,PAY_0,PAY_2,PAY_3,PAY_4,PAY_5,PAY_6,BILL_AMT1,BILL_AMT2,BILL_AMT3,BILL_AMT4,BILL_AMT5,BILL_AMT6,PAY_AMT1,PAY_AMT2,PAY_AMT3,PAY_AMT4,PAY_AMT5,PAY_AMT6
8759,8760,-1.135178,2,1,2,0.0,0,0,0,0,0,-1,-0.558595,-0.582250,-0.599997,-0.598109,-0.554967,-0.672282,-0.270961,-0.292380,-0.347201,-0.152203,-0.260507,-0.306404
29948,29949,0.945432,1,1,1,1.0,-1,-1,-1,-1,0,0,-0.261862,-0.169901,-0.689814,-0.666133,-0.680239,-0.692511,2.131999,-0.343313,-0.231120,-0.355871,-0.356126,-0.356126
29155,29156,-1.135178,2,2,1,3.0,1,2,0,0,0,-2,-0.431853,-0.440174,-0.454388,-0.422791,-0.692511,-0.692511,-0.356126,-0.247758,-0.164888,-0.356126,-0.356126,-0.356126
11522,11523,1.022492,2,1,1,1.0,-1,-1,-1,-1,-1,-1,-0.684204,-0.674036,-0.680158,-0.688250,-0.680401,-0.682424,-0.268794,-0.297735,-0.335982,-0.298882,-0.308444,-0.308444
8242,8243,0.328955,2,2,1,2.0,1,-2,-2,-2,-2,-2,-0.692511,-0.692511,-0.692511,-0.692511,-0.692511,-0.692511,-0.356126,-0.356126,-0.356126,-0.356126,-0.356126,-0.356126
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
29870,29871,-1.212238,1,2,2,0.0,0,0,0,0,0,0,-0.570894,-0.557449,-0.556747,-0.556235,-0.564785,-0.562115,-0.282308,-0.278993,-0.279631,-0.334516,-0.333751,-0.331010
5402,5403,-0.903999,1,1,2,1.0,0,0,0,0,0,0,-0.037643,-0.058560,-0.484354,-0.554482,-0.569532,-0.581427,-0.192108,-0.228634,-0.324253,-0.326866,-0.324253,-0.324253
863,864,-1.212238,2,2,1,1.0,0,0,0,0,0,0,-0.477881,-0.570463,-0.558986,-0.557988,-0.561211,-0.575142,-0.207598,-0.215885,-0.292380,-0.334898,-0.336301,-0.335600
15824,15825,-0.672820,2,2,2,0.0,1,2,2,2,2,-2,-0.637583,-0.640415,-0.583153,-0.586660,-0.692511,-0.692511,-0.356126,-0.069270,-0.354787,-0.356126,-0.356126,-0.356126


# Train & Evaluate: 

Helper Functions:

In [13]:
def print_cm(cm, labels, hide_zeroes=False, hide_diagonal=False, hide_threshold=None):
    """pretty print for confusion matrixes"""
    columnwidth = max([len(x) for x in labels] + [5])  # 5 is value length
    empty_cell = " " * columnwidth
    # Print header
    print("    " + empty_cell, end=" ")
    for label in labels:
        print("%{0}s".format(columnwidth) % label, end=" ")
    print()
    # Print rows
    for i, label1 in enumerate(labels):
        print("    %{0}s".format(columnwidth) % label1, end=" ")
        for j in range(len(labels)):
            cell = "%{0}.1f".format(columnwidth) % cm[i, j]
            if hide_zeroes:
                cell = cell if float(cm[i, j]) != 0 else empty_cell
            if hide_diagonal:
                cell = cell if i != j else empty_cell
            if hide_threshold:
                cell = cell if cm[i, j] > hide_threshold else empty_cell
            print(cell, end=" ")
        print()
        
def train_evaluate_val(model, x_train, y_train, folds, metrics):
    for metric in metrics:
        scores = cross_val_score(model, x_train, y_train, cv=folds, scoring = metric)
        print(f'{metric}:')
        print(f'Cross-validation: {scores}')
        print(f"Average: {scores.mean()}")
        print("----------------------------------------------------------")
    
def test_evaluate(model, x_test, y_test):
    prediction = model.predict(x_test)
    confusion = confusion_matrix(y_test, prediction)
    print("Confusion matrix",print_cm(confusion, ['Not pay', 'pay']))
    print('Accuracy: {:.2f}'.format(accuracy_score(y_test, prediction)))
    print('Precision: {:.2f}'.format(precision_score(y_test, prediction)))
    print('Recall: {:.2f}'.format(recall_score(y_test, prediction)))
    print('F1: {:.2f}'.format(f1_score(y_test, prediction)))
    print('AUC: {:.2f}'.format(roc_auc_score(y_test, prediction)))
    print(classification_report(y_test, prediction, target_names=['not pay', 'pay']))

def optimize_model(model, x_train, y_train, parameters, metrics):
    for metric in metrics:
        print(f'{metric}:')
        grid_clf = GridSearchCV(model, param_grid = parameters, scoring = metric)
        grid_clf.fit(x_train, y_train)
        best_parameter = grid_clf.best_params_
        best_score = grid_clf.best_score_
        print(f"The best parameters: {best_parameter}\nThe best score: {best_score}\n")

In [14]:
metrics = ["accuracy", 'precision', 'recall', 'f1', 'roc_auc']

Logistic Regression:

In [15]:
l_g_grid_values = {'C': [0.001, 0.01, 0.1, 1, 10, 100]}
l_g = LogisticRegression()

train and evaluate using cross validation:

In [16]:
train_evaluate_val(l_g, x_train, y_train, 5, metrics)

accuracy:
Cross-validation: [0.80538735 0.81290457 0.81227814 0.80726665 0.8081019 ]
Average: 0.8091877218626017
----------------------------------------------------------
precision:
Cross-validation: [0.65609756 0.73504274 0.73546512 0.67164179 0.6775    ]
Average: 0.6951494406684381
----------------------------------------------------------
recall:
Cross-validation: [0.25377358 0.24316682 0.23845429 0.25447691 0.25541942]
Average: 0.24905820425728664
----------------------------------------------------------
f1:
Cross-validation: [0.36598639 0.36543909 0.36014235 0.36910458 0.37097878]
Average: 0.3663302396167972
----------------------------------------------------------
roc_auc:
Cross-validation: [0.72152127 0.73963444 0.71084905 0.70806678 0.72435865]
Average: 0.7208860384950748
----------------------------------------------------------


optimize the model using grid search: 

In [17]:
l_g = LogisticRegression()
optimize_model(l_g, x_train, y_train, l_g_grid_values, metrics)

accuracy:
The best parameters: {'C': 1}
The best score: 0.8091877218626017

precision:
The best parameters: {'C': 0.01}
The best score: 0.7024681168341428

recall:
The best parameters: {'C': 1}
The best score: 0.24905820425728664

f1:
The best parameters: {'C': 1}
The best score: 0.3663302396167972

roc_auc:
The best parameters: {'C': 1}
The best score: 0.7208860384950748



Final evaluation on the test data:

In [18]:
l_g = LogisticRegression(C=0.01)
l_g.fit(x_train, y_train)
test_evaluate(l_g, x_test, y_test)

            Not pay     pay 
    Not pay  4510.0   150.0 
        pay  1012.0   315.0 
Confusion matrix None
Accuracy: 0.81
Precision: 0.68
Recall: 0.24
F1: 0.35
AUC: 0.60
              precision    recall  f1-score   support

     not pay       0.82      0.97      0.89      4660
         pay       0.68      0.24      0.35      1327

    accuracy                           0.81      5987
   macro avg       0.75      0.60      0.62      5987
weighted avg       0.79      0.81      0.77      5987



The model can't distinguish between the two classes completly (AUC = 0.5) and predict all the instances as (not payed) == (the majority)

Decision Tree:

In [19]:
d_t = DecisionTreeClassifier()

In [20]:
d_t_grid_values = {'criterion': ["gini", "entropy"],
                   'splitter': ["best", "random"],}

train and evaluate using cross validation:

In [21]:
train_evaluate_val(d_t, x_train, y_train, 5, metrics)

accuracy:
Cross-validation: [0.73209438 0.74086448 0.71705993 0.71747755 0.73042389]
Average: 0.7275840467738568
----------------------------------------------------------
precision:
Cross-validation: [0.3920354  0.4232715  0.36924414 0.3684669  0.40018232]
Average: 0.39064004979358646
----------------------------------------------------------
recall:
Cross-validation: [0.42075472 0.47125353 0.4128181  0.399623   0.40716305]
Average: 0.4223224796827486
----------------------------------------------------------
f1:
Cross-validation: [0.40416855 0.4384411  0.38440367 0.37720488 0.40427509]
Average: 0.4016986600490486
----------------------------------------------------------
roc_auc:
Cross-validation: [0.6174522  0.64711112 0.60721802 0.59954384 0.61090449]
Average: 0.6164459323773176
----------------------------------------------------------


optimize the model using grid search: 

In [22]:
d_t = DecisionTreeClassifier()
                
optimize_model(d_t, x_train, y_train, d_t_grid_values, metrics)

accuracy:
The best parameters: {'criterion': 'entropy', 'splitter': 'best'}
The best score: 0.7293380664021716

precision:
The best parameters: {'criterion': 'entropy', 'splitter': 'best'}
The best score: 0.39631048460004087

recall:
The best parameters: {'criterion': 'entropy', 'splitter': 'best'}
The best score: 0.4153466825529494

f1:
The best parameters: {'criterion': 'entropy', 'splitter': 'best'}
The best score: 0.405636293820505

roc_auc:
The best parameters: {'criterion': 'entropy', 'splitter': 'best'}
The best score: 0.6185014961036254



Final evaluation on the test data:

In [23]:
d_t = DecisionTreeClassifier(criterion = 'gini', splitter = 'best')
d_t.fit(x_train, y_train)
test_evaluate(d_t, x_test, y_test)

            Not pay     pay 
    Not pay  2941.0  1719.0 
        pay   671.0   656.0 
Confusion matrix None
Accuracy: 0.60
Precision: 0.28
Recall: 0.49
F1: 0.35
AUC: 0.56
              precision    recall  f1-score   support

     not pay       0.81      0.63      0.71      4660
         pay       0.28      0.49      0.35      1327

    accuracy                           0.60      5987
   macro avg       0.55      0.56      0.53      5987
weighted avg       0.69      0.60      0.63      5987



Random forest:

In [24]:
r_f = RandomForestClassifier()

In [25]:
r_f_grid_values = {'max_depth': [10, 20, 30, 50, 100],
                    'n_estimators': [10, 20, 40, 80, 100],
                    'criterion': ["gini", "entropy"]}

train and evaluate using cross validation:

In [26]:
train_evaluate_val(r_f, x_train, y_train, 5, metrics)

accuracy:
Cross-validation: [0.81394863 0.82230111 0.81708081 0.80998121 0.82083942]
Average: 0.8168302359574025
----------------------------------------------------------
precision:
Cross-validation: [0.61648177 0.67528271 0.65145985 0.61616162 0.67275748]
Average: 0.6464286868549157
----------------------------------------------------------
recall:
Cross-validation: [0.36886792 0.399623   0.35155514 0.34590009 0.37700283]
Average: 0.36858979602724384
----------------------------------------------------------
f1:
Cross-validation: [0.46622833 0.50684117 0.44746601 0.45294475 0.49004225]
Average: 0.4727044997548311
----------------------------------------------------------
roc_auc:
Cross-validation: [0.76018989 0.78665058 0.76483045 0.74946099 0.77126797]
Average: 0.766479978127976
----------------------------------------------------------


optimize the model using grid search: 

In [ ]:
r_f = RandomForestClassifier()
                
optimize_model(r_f, x_train, y_train, r_f_grid_values, metrics)

accuracy:
The best parameters: {'criterion': 'gini', 'max_depth': 10, 'n_estimators': 80}
The best score: 0.8208811860513677

precision:
The best parameters: {'criterion': 'entropy', 'max_depth': 10, 'n_estimators': 100}
The best score: 0.6841796452839963

recall:
The best parameters: {'criterion': 'gini', 'max_depth': 50, 'n_estimators': 100}
The best score: 0.3697213380043747

f1:
The best parameters: {'criterion': 'gini', 'max_depth': 50, 'n_estimators': 100}
The best score: 0.4723457269106895

roc_auc:
The best parameters: {'criterion': 'entropy', 'max_depth': 10, 'n_estimators': 100}
The best score: 0.7819155679506107



Final evaluation on the test data:

In [ ]:
r_f = RandomForestClassifier(max_depth = 30, n_estimators = 100, criterion = 'gini')
r_f.fit(x_train, y_train)
test_evaluate(r_f, x_test, y_test)

            Not pay     pay 
    Not pay  4375.0   285.0 
        pay  1046.0   281.0 
Confusion matrix None
Accuracy: 0.78
Precision: 0.50
Recall: 0.21
F1: 0.30
AUC: 0.58
              precision    recall  f1-score   support

     not pay       0.81      0.94      0.87      4660
         pay       0.50      0.21      0.30      1327

    accuracy                           0.78      5987
   macro avg       0.65      0.58      0.58      5987
weighted avg       0.74      0.78      0.74      5987



# Deep Neural network:

Metrics:

In [ ]:
from keras import backend as K
from keras.models import Sequential
from keras.layers import Dense

def recall_m(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    recall = true_positives / (possible_positives + K.epsilon())
    return recall

def precision_m(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + K.epsilon())
    return precision

def f1_m(y_true, y_pred):
    precision = precision_m(y_true, y_pred)
    recall = recall_m(y_true, y_pred)
    return 2*((precision*recall)/(precision+recall+K.epsilon()))

In [ ]:
model = Sequential()
model.add(Dense(12, input_dim=24, activation='relu'))
model.add(Dense(20, activation='relu'))
model.add(Dense(30, activation='relu'))
model.add(Dense(20, activation='relu'))
model.add(Dense(1, activation='sigmoid'))
# compile the keras model
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy', precision_m, recall_m, f1_m,])
# fit the keras model on the dataset
model.fit(np.array(x_train), np.array(y_train), epochs=50, batch_size=100, validation_split=0.2)
model.evaluate(x=x_test, y=y_test)


Epoch 1/50
192/192 [==============================] - 14s 4ms/step - loss: 13.4987 - accuracy: 0.6577 - precision_m: 0.0969 - recall_m: 0.2139 - f1_m: 0.0856 - val_loss: 1.5750 - val_accuracy: 0.7807 - val_precision_m: 0.2986 - val_recall_m: 0.0168 - val_f1_m: 0.0315
Epoch 2/50
192/192 [==============================] - 0s 2ms/step - loss: 3.6368 - accuracy: 0.6637 - precision_m: 0.1842 - recall_m: 0.2299 - f1_m: 0.1120 - val_loss: 1.7313 - val_accuracy: 0.7828 - val_precision_m: 0.4861 - val_recall_m: 0.0397 - val_f1_m: 0.0723
Epoch 3/50
192/192 [==============================] - 0s 2ms/step - loss: 4.6758 - accuracy: 0.6693 - precision_m: 0.2496 - recall_m: 0.2431 - f1_m: 0.1290 - val_loss: 8.8013 - val_accuracy: 0.2416 - val_precision_m: 0.2238 - val_recall_m: 0.9849 - val_f1_m: 0.3634
Epoch 4/50
192/192 [==============================] - 0s 2ms/step - loss: 5.3657 - accuracy: 0.6615 - precision_m: 0.2387 - recall_m: 0.2630 - f1_m: 0.1393 - val_loss: 3.5009 - val_accuracy: 0.7816 - 

[0.6184441447257996,
 0.6761316061019897,
 0.36164799332618713,
 0.6058339476585388,
 0.4384687840938568]